# Single Image Processing using Color Attuniation Prior
Github Link: https://github.com/jevonswang/Color-Attenuation-Prior-Dehazing

## Main and Defined Functions:

In [1]:
# Importing Libraries:
import sys,os
import cv2
import numpy as np
import scipy
import scipy.ndimage
import matplotlib.pyplot as plt
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
%load_ext autotime
# from google.colab.patches import cv2_imshow
# !mkdir output

class GuidedFilter:
    
    def __init__(self, I, radius=5, epsilon=0.4):

        self._radius = 2 * radius + 1
        self._epsilon = epsilon
        self._I = self._toFloatImg(I)        
        self._initFilter()

    def _toFloatImg(self, img):
        if img.dtype == np.float32:
            return img
        return ( 1.0 / 255.0 ) * np.float32(img)

    def _initFilter(self):
        I = self._I
        r = self._radius
        eps = self._epsilon
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]

        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                                                     

        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                       

        Irr_inv = Igg_var * Ibb_var - Igb_var * Igb_var                                                      
        Irg_inv = Igb_var * Irb_var - Irg_var * Ibb_var                                                      
        Irb_inv = Irg_var * Igb_var - Igg_var * Irb_var                                                      
        Igg_inv = Irr_var * Ibb_var - Irb_var * Irb_var                                                      
        Igb_inv = Irb_var * Irg_var - Irr_var * Igb_var                                                      
        Ibb_inv = Irr_var * Igg_var - Irg_var * Irg_var                                                      
        
        I_cov = Irr_inv * Irr_var + Irg_inv * Irg_var + Irb_inv * Irb_var                                    
        Irr_inv /= I_cov                                                                                     
        Irg_inv /= I_cov                                                                                     
        Irb_inv /= I_cov                                                                                     
        Igg_inv /= I_cov                                                                                     
        Igb_inv /= I_cov                                                                                     
        Ibb_inv /= I_cov                                                                                     
        
        self._Irr_inv = Irr_inv                                                                              
        self._Irg_inv = Irg_inv                                                                              
        self._Irb_inv = Irb_inv                                                                              
        self._Igg_inv = Igg_inv                                                                              
        self._Igb_inv = Igb_inv                                                                              
        self._Ibb_inv = Ibb_inv                  

    def _computeCoefficients(self, p):
        r = self._radius                                                             
        I = self._I                                                                 
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]                                                          
        
        p_mean = cv2.blur(p, (r, r))                             
        Ipr_mean = cv2.blur(Ir * p, (r, r))                                                         
        Ipg_mean = cv2.blur(Ig * p, (r, r))                                                    
        Ipb_mean = cv2.blur(Ib * p, (r, r))             

        Ipr_cov = Ipr_mean - self._Ir_mean * p_mean                                                 
        Ipg_cov = Ipg_mean - self._Ig_mean * p_mean                                                     
        Ipb_cov = Ipb_mean - self._Ib_mean * p_mean                                                       
                                                                                                                 
        ar = self._Irr_inv * Ipr_cov + self._Irg_inv * Ipg_cov + self._Irb_inv * Ipb_cov                 
        ag = self._Irg_inv * Ipr_cov + self._Igg_inv * Ipg_cov + self._Igb_inv * Ipb_cov                
        ab = self._Irb_inv * Ipr_cov + self._Igb_inv * Ipg_cov + self._Ibb_inv * Ipb_cov    

        b = p_mean - ar * self._Ir_mean - ag * self._Ig_mean - ab * self._Ib_mean                                                                                                                                         

        ar_mean = cv2.blur(ar, (r, r))          
        ag_mean = cv2.blur(ag, (r, r))                                                                   
        ab_mean = cv2.blur(ab, (r, r))                                                                      
        b_mean = cv2.blur(b, (r, r))                                                                                                                                              

        return ar_mean, ag_mean, ab_mean, b_mean            

    def _computeOutput(self, ab, I):    
        ar_mean, ag_mean, ab_mean, b_mean = ab
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]
        q = ar_mean * Ir + ag_mean * Ig + ab_mean * Ib + b_mean
        return q

    def filter(self, p):
        p_32F = self._toFloatImg(p)
        ab = self._computeCoefficients(p)
        return self._computeOutput(ab, self._I)

# Calculated Depth Map:
def calDepthMap(I, r):
    #Calculate the depth map
    hsvI = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)
    s = hsvI[:,:,1] / 255
    v = hsvI[:,:,2] / 255
    #cv2.imshow("hsvI",hsvI)
    #cv2.waitKey()

    sigma = 0.04       # Smaller value of sigma gives more clearer image (tried 0.02) (0.04 Good)
    sigmaMat = np.random.normal(0, sigma, (I.shape[0], I.shape[1]))

    output =  0.121779 + 0.959710 * v - 0.780245 * s + sigmaMat  # Increasign the value of 1st variable (theta_0) increases image quality (tried 0.2) (0.12 Good)
    outputPixel = output
    output = scipy.ndimage.filters.minimum_filter(output,(r,r))
    outputRegion = output
    # cv2.imwrite("/content/output/vsFeature.jpg", outputRegion*255 )
    #cv2.imshow("outputRegion",outputRegion)
    #cv2.waitKey()
    return outputRegion, outputPixel

# Estimated Atmospheric Light:
def estA(img, Jdark):
    #Estimate the value of atmospheric ambient light A

    h,w,c = img.shape
    if img.dtype == np.uint8:
        img = np.float32(img) / 255
    
    # Compute number for 0.1% brightest pixels
    n_bright = int(np.ceil(0.001*h*w))
    #  Loc contains the location of the sorted pixels
    reshaped_Jdark = Jdark.reshape(1,-1)
    Y = np.sort(reshaped_Jdark) 
    Loc = np.argsort(reshaped_Jdark)
    
    # column-stacked version of I
    Ics = img.reshape(1, h*w, 3)
    ix = img.copy()
    dx = Jdark.reshape(1,-1)
    
    # init a matrix to store candidate airlight pixels
    Acand = np.zeros((1, n_bright, 3), dtype=np.float32)
    # init matrix to store largest norm arilight
    Amag = np.zeros((1, n_bright, 1), dtype=np.float32)
    
    # Compute magnitudes of RGB vectors of A
    for i in range(n_bright):
        x = Loc[0,h*w-1-i]
        # print(x,x/w,x%w)
        ix[int(x/w), x%w, 0] = 0
        ix[int(x/w), x%w, 1] = 0
        ix[int(x/w), x%w, 2] = 1
        
        Acand[0, i, :] = Ics[0, Loc[0, h*w-1-i], :]
        Amag[0, i] = np.linalg.norm(Acand[0,i,:])
    
    # Sort A magnitudes
    reshaped_Amag = Amag.reshape(1,-1)
    Y2 = np.sort(reshaped_Amag) 
    Loc2 = np.argsort(reshaped_Amag)
    # A now stores the best estimate of the airlight
    if len(Y2) > 20:
        A = Acand[0, Loc2[0, n_bright-19:n_bright],:]
    else:
        A = Acand[0, Loc2[0,n_bright-len(Y2):n_bright],:]
    
    # finds the max of the 20 brightest pixels in original image
    #print(A)

#     cv2.imshow("brightest",ix)
    #cv2.waitKey()
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Output for Report\position_of_the_atmospheric_light.png", ix*255)
    return A
def notFinal(I):
   save.write(np.uint8(J*255))

def final(I):
    # Other Variables for Image Processing:
    r = 10 #The size of the box when the minimum value is filtered
    beta = 0.2 #scattering coefficient
    gimfiltR = 40 #The size of the radius when guiding the filter
    eps = 10**-2 #The value of epsilon when guiding the filter (Increasing the value highlights more shadows in the image)
    
    dR,dP = calDepthMap(I, r)
    guided_filter = GuidedFilter(I, gimfiltR, eps)
    refineDR = guided_filter.filter(dR)
    tR = np.exp(-beta * refineDR)
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Output for Report\ODM.png", dR*255)
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Output for Report\RDM.png", refineDR*255)
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Output for Report\TM1.png", tR*255)
    a = estA(I, dR)

    if I.dtype == np.uint8:
        I = np.float32(I) / 255

    h,w,c = I.shape
    J = np.zeros((h, w, c), dtype=np.float32)

    J[:,:,0] = I[:,:,0] - a[0,0]
    J[:,:,1] = I[:,:,1] - a[0,1]
    J[:,:,2] = I[:,:,2] - a[0,2]

    t = tR
    t0, t1 = 0.05, 1
    t = t.clip(t0, t1)

    J[:, :, 0] = (J[:, :, 0]  / t) + a[0, 0]
    J[:, :, 1] = (J[:, :, 1]  / t) + a[0, 1]
    J[:, :, 2] = (J[:, :, 2]  / t) + a[0, 2]
    # print(J.shape)
    frameR = np.uint8(J*255)
#     save.write(frameR)
    return frameR

time: 0 ns (started: 2022-12-15 18:22:10 +05:30)


## Single Image Dehazing Code:

In [3]:
if __name__ == "__main__":
    inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\From Sir\dataset\DSC_3074.jpg" #Enter image path
    I = cv2.imread(inputImagePath)
    FI = final(I)
#     cv2.imshow('b',I)
    cv2.imshow('a',FI)
#     Saving the Image:
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Output1_1.png", FI)

    # exit at closing of window
    cv2.waitKey(0) & 0xFF == ord('q')
    cv2.destroyAllWindows()

C:\Users\Pratik\AppData\Local\Temp\ipykernel_15348\2762124366.py:132: DeprecationWarning: Please use `minimum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  output = scipy.ndimage.filters.minimum_filter(output,(r,r))


time: 5.08 s (started: 2022-12-15 18:22:42 +05:30)


## Video Dehazing Code:

In [4]:
if __name__ == "__main__":
    # Video Input:
    inputVideoPath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\Vdo_3.mp4"
    vidObj = cv2.VideoCapture(inputVideoPath)

    # Loop for frames:
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

    FI = None
    for frame_idx in range(int(frame_nos+1)):
        ret, I = vidObj.read()
        
#        Skipping Frames:
#       if (frame_idx % 10 == 0) and ret:
#         FI = final(I)
#       else:
#         save.write(FI)
    
#         Without Skipping Frames:
        if frame_idx and ret:
            FI = final(I)
            save.write(FI)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vidObj.release()
    save.release()
    cv2.destroyAllWindows()
    
print('Execution Completed')

C:\Users\Pratik\AppData\Local\Temp\ipykernel_15348\2762124366.py:132: DeprecationWarning: Please use `minimum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  output = scipy.ndimage.filters.minimum_filter(output,(r,r))


Execution Completed
time: 7min 34s (started: 2022-12-15 18:23:13 +05:30)


## Real-Time Dehazing Code:

In [8]:
if __name__ == "__main__":
#     vidObj_cv = cv2.VideoCapture(0)
    vidObj_vs = VideoStream(0).start()

    time.sleep(1.0)
    start_time = time.time()
    
    # Defining FPS:
    FPS_Start_Time = 0
    FPS = 0
    
    while True:
#         ret, I_cv = vidObj_cv.read()
        I_vs = vidObj_vs.read()
        
        # FPS:
        FPS_End_Time = time.time()
        FPS = 1/(FPS_End_Time - FPS_Start_Time)
        FPS_Start_Time = FPS_End_Time
        FPS_Text = "FPS: {:.2f}".format(FPS)
        cv2.putText(I_vs, FPS_Text, (5,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,0), 1)
        
#         cv2.imshow('Input Sequence',I_cv)
        cv2.imshow('Input Sequence',I_vs)
    
        FI_vs = final(I_vs)
        
#         cv2.imshow('Output Sequence',FI_cv)
        cv2.imshow('Output Sequence',FI_vs)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

#     vidObj_cv.release()
    vidObj_vs.stream.release()
    # save.release()
    cv2.destroyAllWindows()
print("[INFO] Elasped Time: {:.2f} sec".format(time.time() - start_time))

C:\Users\Pratik\AppData\Local\Temp\ipykernel_9836\59830444.py:131: DeprecationWarning: Please use `minimum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  output = scipy.ndimage.filters.minimum_filter(output,(r,r))


[INFO] Elasped Time: 9.88 sec


In [5]:

import sys,os
import cv2
import numpy as np
import scipy
import scipy.ndimage
import matplotlib.pyplot as plt

# from GuidedFilter import GuidedFilter

def calDepthMap(I, r):
    #计算深度图

    hsvI = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)
    s = hsvI[:,:,1] / 255.0
    v = hsvI[:,:,2] / 255.0
    #cv2.imshow("hsvI",hsvI)
    #cv2.waitKey()

    sigma = 0.041337
    sigmaMat = np.random.normal(0, sigma, (I.shape[0], I.shape[1]))

    output =  0.121779 + 0.959710 * v - 0.780245 * s + sigmaMat
    outputPixel = output
    output = scipy.ndimage.filters.minimum_filter(output,(r,r))
    outputRegion = output
#     cv2.imwrite("data/vsFeature.jpg", outputRegion*255 )
    #cv2.imshow("outputRegion",outputRegion)
    #cv2.waitKey()
    return outputRegion, outputPixel

def estA(img, Jdark):
    #估计大气环境光A的值


    h,w,c = img.shape
    if img.dtype == np.uint8:
        img = np.float32(img) / 255
    
    # Compute number for 0.1% brightest pixels
    n_bright = int(np.ceil(0.001*h*w))
    #  Loc contains the location of the sorted pixels
    reshaped_Jdark = Jdark.reshape(1,-1)
    Y = np.sort(reshaped_Jdark) 
    Loc = np.argsort(reshaped_Jdark)
    
    # column-stacked version of I
    Ics = img.reshape(1, h*w, 3)
    ix = img.copy()
    dx = Jdark.reshape(1,-1)
    
    # init a matrix to store candidate airlight pixels
    Acand = np.zeros((1, n_bright, 3), dtype=np.float32)
    # init matrix to store largest norm arilight
    Amag = np.zeros((1, n_bright, 1), dtype=np.float32)
    
    # Compute magnitudes of RGB vectors of A
    for i in range(n_bright):
        x = Loc[0,h*w-1-i]
        ix[int(x/w), x%w, 0] = 0
        ix[int(x/w), x%w, 1] = 0
        ix[int(x/w), x%w, 2] = 1
        
        Acand[0, i, :] = Ics[0, Loc[0, h*w-1-i], :]
        Amag[0, i] = np.linalg.norm(Acand[0,i,:])
    
    # Sort A magnitudes
    reshaped_Amag = Amag.reshape(1,-1)
    Y2 = np.sort(reshaped_Amag) 
    Loc2 = np.argsort(reshaped_Amag)
    # A now stores the best estimate of the airlight
    if len(Y2) > 20:
        A = Acand[0, Loc2[0, n_bright-19:n_bright],:]
    else:
        A = Acand[0, Loc2[0,n_bright-len(Y2):n_bright],:]
    
    # finds the max of the 20 brightest pixels in original image
#     print A

#     cv2.imshow("brightest",ix)
#     cv2.waitKey(0) & 0xFF == ord('q')
#     cv2.destroyAllWindows()
#     cv2.imwrite("data/position_of_the_atmospheric_light.png", ix*255)
    
    return A

if __name__ == "__main__":
    #参数设置
    inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\input.png" #输入图片路径
    r = 15 #最小值滤波时的框的大小
    beta = 1.0 #散射系数
    gimfiltR = 60 #引导滤波时半径的大小
    eps = 10**-3 #引导滤波时epsilon的值


    I = cv2.imread(inputImagePath)
    dR,dP = calDepthMap(I, r)
    guided_filter = GuidedFilter(I, gimfiltR, eps)
    refineDR = guided_filter.filter(dR)
    tR = np.exp(-beta * refineDR)
    tP = np.exp(-beta * dP)

#     cv2.imwrite("data/originalDepthMap.png", dR*255)
#     cv2.imwrite("data/refineDepthMap.png", refineDR*255)
#     cv2.imwrite("data/transmission.png", tR*255)

    a = estA(I, dR)

    if I.dtype == np.uint8:
        I = np.float32(I) / 255

    h,w,c = I.shape
    J = np.zeros((h, w, c), dtype=np.float32)

    J[:,:,0] = I[:,:,0] - a[0,0]
    J[:,:,1] = I[:,:,1] - a[0,1]
    J[:,:,2] = I[:,:,2] - a[0,2]

    t = tR
    t0, t1 = 0.05, 1
    t = t.clip(t0, t1)

    J[:, :, 0] = J[:, :, 0]  / t
    J[:, :, 1] = J[:, :, 1]  / t
    J[:, :, 2] = J[:, :, 2]  / t

    J[:, :, 0] = J[:, :, 0]  + a[0, 0]
    J[:, :, 1] = J[:, :, 1]  + a[0, 1]
    J[:, :, 2] = J[:, :, 2]  + a[0, 2]

#     cv2.imwrite("data/"+str(r)+"_beta"+str(beta)+".png", J*255)

C:\Users\Pratik\AppData\Local\Temp\ipykernel_16544\1823004834.py:24: DeprecationWarning: Please use `minimum_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  output = scipy.ndimage.filters.minimum_filter(output,(r,r))
